<center><h1>Used Car Price Prediction</h1></center>

# Import Libraries 

In [2]:
from sklearn.model_selection import train_test_split 
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor


from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV,KFold,RandomizedSearchCV

from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer,SimpleImputer
import matplotlib.pyplot as plt
import matplotlib

from pymongo import MongoClient

# Read data from MongoDB

In [3]:
from pymongo import MongoClient
  
# Create a pymongo client
client = MongoClient('localhost', 27017)
  
# database instance
db = client['CarData']
  
# collection instance
doc = db['car']
  
# Retrieving first 3 documents using the
# find() and limit() methods 
l1 =[]
for doc1 in doc.find().limit(100000):
    l1.append(doc1)
    #print(doc1)



In [4]:
df = pd.DataFrame(l1)
df

,_id,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,lat,long,price
0,62409de88d9a212b80d2832f,7119330988,2012.0,fiat,excellent,4 cylinders,gas,142000.0,clean,automatic,fwd,full-size,offroad,grey,30.8464,-85.6900,4500.0
1,62409de88d9a212b80d28330,7109175069,2006.0,toyota,good,6 cylinders,gas,159009.0,clean,automatic,4wd,full-size,van,silver,38.4947,-77.9926,6475.0
2,62409de88d9a212b80d28331,7109174154,2003.0,toyota,good,6 cylinders,gas,151089.0,clean,automatic,4wd,full-size,SUV,red,38.4947,-77.9926,4995.0
3,62409de88d9a212b80d28332,7109173283,1998.0,mercedes-benz,good,6 cylinders,gas,88367.0,clean,automatic,4wd,full-size,SUV,silver,38.4947,-77.9926,4500.0
4,62409de88d9a212b80d28333,7109172375,2007.0,lexus,good,6 cylinders,gas,169770.0,clean,automatic,4wd,full-size,SUV,green,38.4947,-77.9926,6995.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,62409dea8d9a212b80d407ee,7121385313,2011.0,ford,fair,5 cylinders,gas,147075.0,clean,automatic,4wd,full-size,SUV,red,40.1352,-74.7036,7399.0
99996,62409dea8d9a212b80d407ef,7121360167,2013.0,ford,fair,5 cylinders,gas,110875.0,clean,automatic,4wd,full-size,pickup,grey,40.1352,-74.7036,10999.0
99997,62409dea8d9a212b80d407f0,7118198890,2016.0,honda,fair,4 cylinders,gas,40016.0,clean,automatic,fwd,full-size,wagon,black,34.0627,-117.6720,12500.0
99998,62409dea8d9a212b80d407f1,7121333014,2016.0,dodge,fair,5 cylinders,gas,35947.0,clean,automatic,4wd,full-size,hatchback,grey,40.1352,-74.7036,12899.0


In [5]:
df = df.drop(["_id"],axis=True)
df = pd.get_dummies(df,drop_first=True)
df.head(5)

,id,year,odometer,lat,long,price,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,...,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_truck,paint_color_white,paint_color_yellow
0,7119330988,2012.0,142000.0,30.8464,-85.6900,4500.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,7109175069,2006.0,159009.0,38.4947,-77.9926,6475.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,7109174154,2003.0,151089.0,38.4947,-77.9926,4995.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,7109173283,1998.0,88367.0,38.4947,-77.9926,4500.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,7109172375,2007.0,169770.0,38.4947,-77.9926,6995.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [6]:
df.shape

(100000, 105)

In [7]:
print(df.isnull().sum())
print(df.shape)

id                    0
year                  0
odometer              2
lat                   2
long                  0
                     ..
paint_color_red       0
paint_color_silver    0
paint_color_truck     0
paint_color_white     0
paint_color_yellow    0
Length: 105, dtype: int64
(100000, 105)


In [8]:
X=df.drop(["id","price"],axis=True)
y=df["price"]
print(X.shape)
print(y.shape)
print(X.columns)
print(y)

(100000, 103)
(100000,)
Index(['year', 'odometer', 'lat', 'long', 'manufacturer_alfa-romeo',
       'manufacturer_aston-martin', 'manufacturer_audi', 'manufacturer_bmw',
       'manufacturer_buick', 'manufacturer_cadillac',
       ...
       'paint_color_custom', 'paint_color_green', 'paint_color_grey',
       'paint_color_orange', 'paint_color_purple', 'paint_color_red',
       'paint_color_silver', 'paint_color_truck', 'paint_color_white',
       'paint_color_yellow'],
      dtype='object', length=103)
0         4500.0
1         6475.0
2         4995.0
3         4500.0
4         6995.0
          ...   
99995     7399.0
99996    10999.0
99997    12500.0
99998    12899.0
99999    12999.0
Name: price, Length: 100000, dtype: float64


# calculate missing value- using Imputer

In [9]:
imputer = SimpleImputer(strategy='mean')
X =imputer.fit_transform(X) 

In [ ]:
print(X.isnull().sum())
print(X.shape)

# Train Test split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=2021)

# Linear Regression

In [15]:
#Grid search cv
kfold = KFold(n_splits=5, random_state=2020,shuffle=True)
LR = LinearRegression()
results = cross_val_score(LR, X_train,y_train, cv=kfold)
print(results)
print("R2: %.4f (%.4f)" % (results.mean(), results.std()))

[0.62172227 0.61272732 0.62387275 0.61569704 0.62126806]
R2: 0.6191 (0.0042)


In [18]:
#With Train test split
LR=LinearRegression()
LR.fit(X_train,y_train)
y_pred=LR.predict(X_test)
print("r2_score = ",r2_score(y_test, y_pred))

r2_score =  0.6193325846505515


In [24]:
cols = pd.DataFrame(X, columns =df.drop(["id","price"],axis=True).columns)

# Ridge Regression

In [26]:
#With Grid Search CV
kfold= KFold(n_splits=5,random_state=2021,shuffle=True)
model = Ridge()
parameters= {'alpha': [0.001,0.1,0.2,0.3,0.4,0.5]}
#fit the grid search
cv= GridSearchCV(model,parameters,scoring='r2',cv=kfold)
cv.fit(X_train,y_train)

#Best Parameters
print(cv.best_params_)
print(cv.best_score_)

y_pred = cv.predict(X_test)
print("r2_score = ",r2_score(y_test, y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.54576e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.28643e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.28746e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.26607e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAl

{'alpha': 0.5}
0.6184390929880476
r2_score =  0.6192666442558455


`Ridge Regression
{'alpha': 0.5}
0.6184390929880476`
* r2_score =  0.6192666442558455

# Lasso Regression

In [27]:
#Grid search cv
parameters = dict(alpha=np.linspace(0.001,10))
model = Lasso()
kfold = KFold(n_splits=5, random_state=2021,shuffle=True)
cv = GridSearchCV(model, param_grid=parameters,
                  cv=kfold,scoring='r2')

cv.fit(X_train,y_train)

#Best Parameters
print(cv.best_params_)
print(cv.best_score_)

y_pred = cv.predict(X_test)
print("r2_score = ",r2_score(y_test, y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.376e+12, tolerance: 7.265e+08
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.376e+12, tolerance: 7.214e+08
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.346e+12, toleranc

{'alpha': 0.20506122448979594}
0.6184460017625509
r2_score =  0.6191765085317995


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.272e+12, tolerance: 9.017e+08
  model = cd_fast.enet_coordinate_descent(


`LASSO Regression
{'alpha': 0.20506122448979594}
0.6184460017625509`
* r2_score =  0.6191765085317995

# Knn Regressor

In [11]:
knn= KNeighborsRegressor()
kfold=KFold(n_splits=5,random_state=2021,shuffle=True)
parameter={"n_neighbors":[1,3,5,7,9,11,13,15]}
cv=GridSearchCV(estimator=knn, param_grid=parameter,
                scoring="r2",cv=kfold,verbose=3)

cv.fit(X_train,y_train)
y_pred = cv.predict(X_test)
print("r2_score = ",r2_score(y_test, y_pred))


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .....................n_neighbors=1;, score=0.111 total time=  20.1s
[CV 2/5] END .....................n_neighbors=1;, score=0.150 total time=  18.6s
[CV 3/5] END .....................n_neighbors=1;, score=0.087 total time=  17.4s
[CV 4/5] END .....................n_neighbors=1;, score=0.119 total time=  19.0s
[CV 5/5] END .....................n_neighbors=1;, score=0.112 total time=  18.6s
[CV 1/5] END .....................n_neighbors=3;, score=0.293 total time=  24.7s
[CV 2/5] END .....................n_neighbors=3;, score=0.307 total time=  25.6s
[CV 3/5] END .....................n_neighbors=3;, score=0.257 total time=  25.7s
[CV 4/5] END .....................n_neighbors=3;, score=0.279 total time=  25.7s
[CV 5/5] END .....................n_neighbors=3;, score=0.287 total time=  26.0s
[CV 1/5] END .....................n_neighbors=5;, score=0.320 total time=  31.5s
[CV 2/5] END .....................n_neighbors=5;,

`KNN` 
* r2_score =  0.33049468231521995


# Random Forest Regressor

In [11]:
#Grid search cv
model = RandomForestRegressor(random_state=2021)
kfold = KFold(n_splits=5, random_state=2021,shuffle=True)
parameters={'max_features':[6,7,8],
           'n_estimators':[100,150,180],
           'min_samples_leaf':[1,3,5]}

cv=GridSearchCV(estimator=model, param_grid=parameters,
                cv=kfold,verbose=3)

cv.fit(X,y)
print(cv.best_params_)
print(cv.best_score_)

bm_ridge = cv.best_estimator_

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END max_features=6, min_samples_leaf=1, n_estimators=100; total time=  47.6s
[CV 2/5] END max_features=6, min_samples_leaf=1, n_estimators=100; total time=  39.6s
[CV 3/5] END max_features=6, min_samples_leaf=1, n_estimators=100; total time=  35.2s
[CV 4/5] END max_features=6, min_samples_leaf=1, n_estimators=100; total time=  38.4s
[CV 5/5] END max_features=6, min_samples_leaf=1, n_estimators=100; total time=  38.7s
[CV 1/5] END max_features=6, min_samples_leaf=1, n_estimators=150; total time=  56.3s
[CV 2/5] END max_features=6, min_samples_leaf=1, n_estimators=150; total time=  57.1s
[CV 3/5] END max_features=6, min_samples_leaf=1, n_estimators=150; total time=  58.4s
[CV 4/5] END max_features=6, min_samples_leaf=1, n_estimators=150; total time=  55.5s
[CV 5/5] END max_features=6, min_samples_leaf=1, n_estimators=150; total time=  53.8s
[CV 1/5] END max_features=6, min_samples_leaf=1, n_estimators=180; total time=

[CV 1/5] END max_features=8, min_samples_leaf=1, n_estimators=150; total time=  47.6s
[CV 2/5] END max_features=8, min_samples_leaf=1, n_estimators=150; total time=  46.9s
[CV 3/5] END max_features=8, min_samples_leaf=1, n_estimators=150; total time=  45.1s
[CV 4/5] END max_features=8, min_samples_leaf=1, n_estimators=150; total time=  44.4s
[CV 5/5] END max_features=8, min_samples_leaf=1, n_estimators=150; total time=  44.3s
[CV 1/5] END max_features=8, min_samples_leaf=1, n_estimators=180; total time= 1.0min
[CV 2/5] END max_features=8, min_samples_leaf=1, n_estimators=180; total time= 1.0min
[CV 3/5] END max_features=8, min_samples_leaf=1, n_estimators=180; total time=  59.9s
[CV 4/5] END max_features=8, min_samples_leaf=1, n_estimators=180; total time=  55.6s
[CV 5/5] END max_features=8, min_samples_leaf=1, n_estimators=180; total time=  49.7s
[CV 1/5] END max_features=8, min_samples_leaf=3, n_estimators=100; total time=  16.3s
[CV 2/5] END max_features=8, min_samples_leaf=3, n_est

`Random Forest
{'max_features': 8, 'min_samples_leaf': 1, 'n_estimators': 180}
0.8184960038334529`
* r2_score =  0.8184960038334529


# Bagging Regressor

In [32]:
#Grid search cv
model = BaggingRegressor(random_state=2021)
kfold= KFold(n_splits=5,random_state=2021,shuffle=True)
parameters= {'n_estimators': [15,20,30,40],'max_features':[5,8,10,15]}
#fit the grid search
cv= GridSearchCV(model,parameters,scoring='r2',cv=kfold,verbose=3)
cv.fit(X_train,y_train)

#Best Parameters
print(cv.best_params_)
print(cv.best_score_)

y_pred = cv.predict(X_test)
print("r2_score = ",r2_score(y_test, y_pred))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ...max_features=5, n_estimators=15;, score=0.134 total time=   0.2s
[CV 2/5] END ...max_features=5, n_estimators=15;, score=0.128 total time=   0.2s
[CV 3/5] END ...max_features=5, n_estimators=15;, score=0.126 total time=   0.2s
[CV 4/5] END ...max_features=5, n_estimators=15;, score=0.128 total time=   0.2s
[CV 5/5] END ...max_features=5, n_estimators=15;, score=0.132 total time=   0.2s
[CV 1/5] END ...max_features=5, n_estimators=20;, score=0.184 total time=   0.3s
[CV 2/5] END ...max_features=5, n_estimators=20;, score=0.180 total time=   0.3s
[CV 3/5] END ...max_features=5, n_estimators=20;, score=0.174 total time=   0.3s
[CV 4/5] END ...max_features=5, n_estimators=20;, score=0.179 total time=   0.3s
[CV 5/5] END ...max_features=5, n_estimators=20;, score=0.181 total time=   0.3s
[CV 1/5] END ...max_features=5, n_estimators=30;, score=0.228 total time=   0.6s
[CV 2/5] END ...max_features=5, n_estimators=30;

`Bagging Regresor
{'max_features': 15, 'n_estimators': 40}
0.4218406711762924
r2_score =  0.42881801551338306`

# AdaBoost Regressor

In [33]:
#Grid Search cv
model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=24),random_state=2021)
kfold= KFold(n_splits=5,random_state=2021,shuffle=True)
cv=GridSearchCV(cv=kfold,estimator=model,
                 param_grid={'learning_rate': [0.2, 0.3,0.6],
                'n_estimators': [60, 70, 80, 90,100]},
                scoring='r2',verbose=3)

cv.fit(X_train,y_train)

#Best Parameters
print(cv.best_params_)
print(cv.best_score_)

y_pred = cv.predict(X_test)
print("r2_score = ",r2_score(y_test, y_pred))

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END learning_rate=0.2, n_estimators=60;, score=0.801 total time= 1.0min
[CV 2/5] END learning_rate=0.2, n_estimators=60;, score=0.806 total time= 1.0min
[CV 3/5] END learning_rate=0.2, n_estimators=60;, score=0.774 total time= 1.0min
[CV 4/5] END learning_rate=0.2, n_estimators=60;, score=0.758 total time= 1.1min
[CV 5/5] END learning_rate=0.2, n_estimators=60;, score=0.805 total time= 1.1min
[CV 1/5] END learning_rate=0.2, n_estimators=70;, score=0.802 total time= 1.2min
[CV 2/5] END learning_rate=0.2, n_estimators=70;, score=0.804 total time= 1.6min
[CV 3/5] END learning_rate=0.2, n_estimators=70;, score=0.772 total time= 1.4min
[CV 4/5] END learning_rate=0.2, n_estimators=70;, score=0.764 total time= 1.4min
[CV 5/5] END learning_rate=0.2, n_estimators=70;, score=0.803 total time= 1.3min
[CV 1/5] END learning_rate=0.2, n_estimators=80;, score=0.803 total time= 1.4min
[CV 2/5] END learning_rate=0.2, n_estimators=80;

` AdaBoost Regressor
{'learning_rate': 0.6, 'n_estimators': 100}
0.8005646719615527
r2_score =  0.7992880899482`

# XGBoost

In [12]:
#Grid Search cv
lr_range = [0.001, 0.01, 0.1, 0.2,0.25, 0.3]
n_est_range = [10,20,30,50,100]
md_range = [2,4,6,8,10]

parameters = {'learning_rate':lr_range,'n_estimators':n_est_range,
                'max_depth':md_range}
kfold = KFold(n_splits=5, random_state=2021,shuffle=True)

clf = XGBRegressor(random_state=2021)
cv = GridSearchCV(clf, param_grid=parameters,
                  cv=kfold,scoring='r2')


cv.fit(X_train,y_train)
y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)

print("r2_score = ",r2_score(y_test, y_pred))

{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 100}
0.7968035903222808
r2_score =  0.8079557972846251


`XG Boost
{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 100}
0.7968035903222808`
* r2_score =  0.8079557972846251

# CatBoostRegressor

In [13]:
cbr = CatBoostRegressor(learning_rate=0.03,random_state=2021)
cbr.fit(X_train,y_train,verbose=True,plot=True)
y_pred = cbr.predict(X_test)
print("r2_score = ",r2_score(y_test, y_pred))


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 11138.9268154	total: 162ms	remaining: 2m 41s
1:	learn: 10969.8490010	total: 170ms	remaining: 1m 24s
2:	learn: 10803.8506510	total: 177ms	remaining: 58.8s
3:	learn: 10644.3516473	total: 185ms	remaining: 45.9s
4:	learn: 10494.8814844	total: 192ms	remaining: 38.2s
5:	learn: 10349.7592056	total: 200ms	remaining: 33.1s
6:	learn: 10215.7686744	total: 207ms	remaining: 29.4s
7:	learn: 10080.7606501	total: 215ms	remaining: 26.6s
8:	learn: 9952.4694442	total: 222ms	remaining: 24.4s
9:	learn: 9832.7735688	total: 229ms	remaining: 22.7s
10:	learn: 9716.6466885	total: 236ms	remaining: 21.2s
11:	learn: 9603.8272205	total: 244ms	remaining: 20.1s
12:	learn: 9492.8636224	total: 252ms	remaining: 19.2s
13:	learn: 9390.2066152	total: 264ms	remaining: 18.6s
14:	learn: 9291.3528171	total: 272ms	remaining: 17.9s
15:	learn: 9194.8366383	total: 280ms	remaining: 17.2s
16:	learn: 9102.9943484	total: 288ms	remaining: 16.7s
17:	learn: 9018.0840197	total: 296ms	remaining: 16.1s
18:	learn: 8934.9626165	tota

153:	learn: 6473.5890557	total: 2.53s	remaining: 13.9s
154:	learn: 6470.9107668	total: 2.55s	remaining: 13.9s
155:	learn: 6467.9442693	total: 2.57s	remaining: 13.9s
156:	learn: 6464.4743680	total: 2.58s	remaining: 13.9s
157:	learn: 6461.3814104	total: 2.6s	remaining: 13.9s
158:	learn: 6457.2333247	total: 2.62s	remaining: 13.8s
159:	learn: 6453.5378008	total: 2.63s	remaining: 13.8s
160:	learn: 6450.6968302	total: 2.65s	remaining: 13.8s
161:	learn: 6447.4133181	total: 2.66s	remaining: 13.8s
162:	learn: 6444.4999883	total: 2.68s	remaining: 13.8s
163:	learn: 6440.9639706	total: 2.69s	remaining: 13.7s
164:	learn: 6437.2885582	total: 2.71s	remaining: 13.7s
165:	learn: 6434.3877377	total: 2.73s	remaining: 13.7s
166:	learn: 6431.6886992	total: 2.74s	remaining: 13.7s
167:	learn: 6426.5278068	total: 2.76s	remaining: 13.7s
168:	learn: 6423.9775689	total: 2.77s	remaining: 13.6s
169:	learn: 6420.7438427	total: 2.79s	remaining: 13.6s
170:	learn: 6416.1536950	total: 2.81s	remaining: 13.6s
171:	learn:

313:	learn: 6060.7304192	total: 4.5s	remaining: 9.82s
314:	learn: 6058.5352984	total: 4.5s	remaining: 9.79s
315:	learn: 6056.6835959	total: 4.51s	remaining: 9.77s
316:	learn: 6054.5961840	total: 4.52s	remaining: 9.74s
317:	learn: 6053.5779780	total: 4.53s	remaining: 9.71s
318:	learn: 6051.9332887	total: 4.53s	remaining: 9.68s
319:	learn: 6050.4525769	total: 4.54s	remaining: 9.64s
320:	learn: 6048.8093479	total: 4.54s	remaining: 9.61s
321:	learn: 6047.0756609	total: 4.55s	remaining: 9.58s
322:	learn: 6044.8041613	total: 4.56s	remaining: 9.55s
323:	learn: 6043.5998444	total: 4.56s	remaining: 9.52s
324:	learn: 6041.5889579	total: 4.57s	remaining: 9.49s
325:	learn: 6040.7746373	total: 4.58s	remaining: 9.46s
326:	learn: 6038.7074944	total: 4.58s	remaining: 9.43s
327:	learn: 6037.0957284	total: 4.59s	remaining: 9.4s
328:	learn: 6035.0413150	total: 4.6s	remaining: 9.38s
329:	learn: 6033.6305461	total: 4.6s	remaining: 9.35s
330:	learn: 6032.1589741	total: 4.61s	remaining: 9.32s
331:	learn: 603

475:	learn: 5833.0846865	total: 6.21s	remaining: 6.83s
476:	learn: 5832.4123189	total: 6.22s	remaining: 6.82s
477:	learn: 5831.0399223	total: 6.24s	remaining: 6.81s
478:	learn: 5829.5642733	total: 6.25s	remaining: 6.8s
479:	learn: 5827.6382581	total: 6.27s	remaining: 6.79s
480:	learn: 5826.5884816	total: 6.28s	remaining: 6.78s
481:	learn: 5825.1974263	total: 6.29s	remaining: 6.76s
482:	learn: 5823.5630417	total: 6.31s	remaining: 6.75s
483:	learn: 5822.1747243	total: 6.32s	remaining: 6.74s
484:	learn: 5821.2495720	total: 6.34s	remaining: 6.73s
485:	learn: 5820.2011587	total: 6.35s	remaining: 6.72s
486:	learn: 5818.6841655	total: 6.37s	remaining: 6.71s
487:	learn: 5817.5714484	total: 6.38s	remaining: 6.69s
488:	learn: 5816.4402156	total: 6.4s	remaining: 6.68s
489:	learn: 5815.4550556	total: 6.41s	remaining: 6.67s
490:	learn: 5814.5589120	total: 6.42s	remaining: 6.66s
491:	learn: 5814.0135811	total: 6.44s	remaining: 6.65s
492:	learn: 5812.7779348	total: 6.45s	remaining: 6.64s
493:	learn: 

632:	learn: 5671.1318219	total: 8.62s	remaining: 5s
633:	learn: 5670.0729953	total: 8.63s	remaining: 4.98s
634:	learn: 5668.5246294	total: 8.64s	remaining: 4.96s
635:	learn: 5667.6926661	total: 8.64s	remaining: 4.95s
636:	learn: 5666.5464899	total: 8.65s	remaining: 4.93s
637:	learn: 5666.1310905	total: 8.66s	remaining: 4.91s
638:	learn: 5665.6038604	total: 8.67s	remaining: 4.9s
639:	learn: 5664.8799941	total: 8.68s	remaining: 4.88s
640:	learn: 5664.0832543	total: 8.68s	remaining: 4.86s
641:	learn: 5663.2263814	total: 8.69s	remaining: 4.85s
642:	learn: 5662.7137516	total: 8.7s	remaining: 4.83s
643:	learn: 5661.7804578	total: 8.71s	remaining: 4.81s
644:	learn: 5660.6731413	total: 8.71s	remaining: 4.8s
645:	learn: 5659.6553049	total: 8.72s	remaining: 4.78s
646:	learn: 5658.7086706	total: 8.73s	remaining: 4.76s
647:	learn: 5658.2146354	total: 8.74s	remaining: 4.75s
648:	learn: 5657.5844185	total: 8.75s	remaining: 4.73s
649:	learn: 5656.8450238	total: 8.76s	remaining: 4.71s
650:	learn: 5656

789:	learn: 5544.9434721	total: 10.4s	remaining: 2.77s
790:	learn: 5544.3895347	total: 10.4s	remaining: 2.76s
791:	learn: 5543.4890039	total: 10.5s	remaining: 2.75s
792:	learn: 5542.5267248	total: 10.5s	remaining: 2.74s
793:	learn: 5541.9127996	total: 10.5s	remaining: 2.72s
794:	learn: 5541.1436531	total: 10.5s	remaining: 2.71s
795:	learn: 5540.5496183	total: 10.5s	remaining: 2.7s
796:	learn: 5540.0147343	total: 10.6s	remaining: 2.69s
797:	learn: 5539.5235173	total: 10.6s	remaining: 2.68s
798:	learn: 5538.3508329	total: 10.6s	remaining: 2.66s
799:	learn: 5537.4677080	total: 10.6s	remaining: 2.65s
800:	learn: 5536.8769908	total: 10.6s	remaining: 2.64s
801:	learn: 5535.7520747	total: 10.6s	remaining: 2.63s
802:	learn: 5535.6347856	total: 10.7s	remaining: 2.62s
803:	learn: 5534.9085507	total: 10.7s	remaining: 2.6s
804:	learn: 5534.6662428	total: 10.7s	remaining: 2.59s
805:	learn: 5534.1517276	total: 10.7s	remaining: 2.58s
806:	learn: 5533.3930689	total: 10.7s	remaining: 2.57s
807:	learn: 

951:	learn: 5443.9842967	total: 13.1s	remaining: 660ms
952:	learn: 5442.7688539	total: 13.1s	remaining: 646ms
953:	learn: 5442.5719691	total: 13.1s	remaining: 632ms
954:	learn: 5442.0572268	total: 13.1s	remaining: 618ms
955:	learn: 5441.5303177	total: 13.1s	remaining: 604ms
956:	learn: 5440.9435552	total: 13.1s	remaining: 590ms
957:	learn: 5440.5542146	total: 13.1s	remaining: 576ms
958:	learn: 5439.8329041	total: 13.1s	remaining: 562ms
959:	learn: 5439.4437634	total: 13.2s	remaining: 548ms
960:	learn: 5437.9892489	total: 13.2s	remaining: 534ms
961:	learn: 5437.5468629	total: 13.2s	remaining: 520ms
962:	learn: 5437.0718619	total: 13.2s	remaining: 506ms
963:	learn: 5436.8183233	total: 13.2s	remaining: 493ms
964:	learn: 5436.2902990	total: 13.2s	remaining: 479ms
965:	learn: 5435.9403191	total: 13.2s	remaining: 465ms
966:	learn: 5435.2942956	total: 13.2s	remaining: 451ms
967:	learn: 5434.9282447	total: 13.2s	remaining: 437ms
968:	learn: 5434.4926875	total: 13.2s	remaining: 423ms
969:	learn

`CAT Boost`
* r2_score =  0.7510773791747923

# Hyper-parameter Tuning

In [14]:
cbr = CatBoostRegressor(random_state=2021,iterations=250)
grid = {
    'learning_rate': [0.03, 0.1],
    'depth':[4, 6, 10],
    'l2_leaf_reg': [1, 3, 5]}
cbr.grid_search(grid,X_train,y_train, verbose=3, plot=True)
y_pred =cbr.predict(X_test)
print("r2_score = ",r2_score(y_test, y_pred))


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 17900.3057863	test: 17984.6840070	best: 17984.6840070 (0)	total: 8.4ms	remaining: 2.09s
1:	learn: 17477.5548615	test: 17562.2470855	best: 17562.2470855 (1)	total: 12.7ms	remaining: 1.57s
2:	learn: 17080.3450506	test: 17163.0312108	best: 17163.0312108 (2)	total: 16.8ms	remaining: 1.39s
3:	learn: 16685.0992196	test: 16768.5082146	best: 16768.5082146 (3)	total: 21.2ms	remaining: 1.3s
4:	learn: 16314.7959164	test: 16397.7248979	best: 16397.7248979 (4)	total: 26.2ms	remaining: 1.28s
5:	learn: 15951.9805317	test: 16034.6644464	best: 16034.6644464 (5)	total: 30.7ms	remaining: 1.25s
6:	learn: 15607.3009407	test: 15690.8617279	best: 15690.8617279 (6)	total: 35.2ms	remaining: 1.22s
7:	learn: 15264.1308928	test: 15348.0135824	best: 15348.0135824 (7)	total: 39.6ms	remaining: 1.2s
8:	learn: 14930.9982418	test: 15014.8858314	best: 15014.8858314 (8)	total: 43.9ms	remaining: 1.18s
9:	learn: 14620.6410946	test: 14704.5244915	best: 14704.5244915 (9)	total: 48.2ms	remaining: 1.16s
10:	learn: 14

84:	learn: 7311.4492811	test: 7378.7861599	best: 7378.7861599 (84)	total: 775ms	remaining: 1.5s
85:	learn: 7296.4914479	test: 7363.7146229	best: 7363.7146229 (85)	total: 784ms	remaining: 1.49s
86:	learn: 7282.5305625	test: 7348.9208281	best: 7348.9208281 (86)	total: 794ms	remaining: 1.49s
87:	learn: 7266.7719775	test: 7332.2524446	best: 7332.2524446 (87)	total: 804ms	remaining: 1.48s
88:	learn: 7252.6916314	test: 7316.8862039	best: 7316.8862039 (88)	total: 813ms	remaining: 1.47s
89:	learn: 7238.6295063	test: 7304.3801122	best: 7304.3801122 (89)	total: 834ms	remaining: 1.48s
90:	learn: 7226.6810318	test: 7294.1851365	best: 7294.1851365 (90)	total: 843ms	remaining: 1.47s
91:	learn: 7212.6754392	test: 7280.7997376	best: 7280.7997376 (91)	total: 855ms	remaining: 1.47s
92:	learn: 7200.7863652	test: 7270.8706284	best: 7270.8706284 (92)	total: 873ms	remaining: 1.47s
93:	learn: 7188.2218945	test: 7258.7300169	best: 7258.7300169 (93)	total: 884ms	remaining: 1.47s
94:	learn: 7177.0850042	test: 7

178:	learn: 6669.1275718	test: 6732.5388926	best: 6732.5388926 (178)	total: 1.74s	remaining: 689ms
179:	learn: 6665.9708528	test: 6729.0363809	best: 6729.0363809 (179)	total: 1.75s	remaining: 679ms
180:	learn: 6663.3248373	test: 6726.1951550	best: 6726.1951550 (180)	total: 1.76s	remaining: 670ms
181:	learn: 6660.5949441	test: 6723.5445061	best: 6723.5445061 (181)	total: 1.77s	remaining: 660ms
182:	learn: 6657.1982806	test: 6719.5509251	best: 6719.5509251 (182)	total: 1.78s	remaining: 651ms
183:	learn: 6654.5381241	test: 6716.6688964	best: 6716.6688964 (183)	total: 1.79s	remaining: 641ms
184:	learn: 6649.6300115	test: 6711.7149589	best: 6711.7149589 (184)	total: 1.8s	remaining: 632ms
185:	learn: 6646.9786644	test: 6708.1614235	best: 6708.1614235 (185)	total: 1.8s	remaining: 621ms
186:	learn: 6644.5189846	test: 6705.5603324	best: 6705.5603324 (186)	total: 1.81s	remaining: 611ms
187:	learn: 6640.7511447	test: 6702.5267021	best: 6702.5267021 (187)	total: 1.82s	remaining: 601ms
188:	learn: 

20:	learn: 7566.6704695	test: 7635.2177551	best: 7635.2177551 (20)	total: 243ms	remaining: 2.65s
21:	learn: 7492.4052000	test: 7558.3062815	best: 7558.3062815 (21)	total: 254ms	remaining: 2.64s
22:	learn: 7429.1094474	test: 7488.7891454	best: 7488.7891454 (22)	total: 264ms	remaining: 2.6s
23:	learn: 7372.4044136	test: 7437.1620338	best: 7437.1620338 (23)	total: 278ms	remaining: 2.62s
24:	learn: 7315.8108659	test: 7378.5715605	best: 7378.5715605 (24)	total: 288ms	remaining: 2.59s
25:	learn: 7266.0959873	test: 7330.4781935	best: 7330.4781935 (25)	total: 298ms	remaining: 2.57s
26:	learn: 7220.4015109	test: 7282.2676486	best: 7282.2676486 (26)	total: 310ms	remaining: 2.56s
27:	learn: 7182.1158033	test: 7241.2393065	best: 7241.2393065 (27)	total: 319ms	remaining: 2.53s
28:	learn: 7143.1635383	test: 7200.1934179	best: 7200.1934179 (28)	total: 330ms	remaining: 2.51s
29:	learn: 7106.7999687	test: 7162.7579380	best: 7162.7579380 (29)	total: 339ms	remaining: 2.48s
30:	learn: 7069.6381847	test: 7

...

121:	learn: 5211.8696750	test: 5460.8054968	best: 5460.8054968 (121)	total: 4.75s	remaining: 4.99s
122:	learn: 5205.1453235	test: 5456.8824866	best: 5456.8824866 (122)	total: 4.77s	remaining: 4.93s
123:	learn: 5202.3616069	test: 5454.6474017	best: 5454.6474017 (123)	total: 4.79s	remaining: 4.87s
124:	learn: 5193.5181663	test: 5451.0265467	best: 5451.0265467 (124)	total: 4.81s	remaining: 4.81s
125:	learn: 5186.0031674	test: 5447.8911891	best: 5447.8911891 (125)	total: 4.83s	remaining: 4.75s
126:	learn: 5178.5485912	test: 5443.7505965	best: 5443.7505965 (126)	total: 4.85s	remaining: 4.7s
127:	learn: 5169.3418174	test: 5441.6265343	best: 5441.6265343 (127)	total: 4.88s	remaining: 4.65s
128:	learn: 5159.8259305	test: 5437.7241993	best: 5437.7241993 (128)	total: 4.9s	remaining: 4.59s
129:	learn: 5150.0579309	test: 5431.5222944	best: 5431.5222944 (129)	total: 4.92s	remaining: 4.54s
130:	learn: 5144.7614261	test: 5426.0044909	best: 5426.0044909 (130)	total: 4.94s	remaining: 4.49s
131:	learn: 

205:	learn: 4708.0813393	test: 5241.1071279	best: 5241.1071279 (205)	total: 7.36s	remaining: 1.57s
206:	learn: 4704.9843193	test: 5239.3010443	best: 5239.3010443 (206)	total: 7.4s	remaining: 1.54s
207:	learn: 4697.8344966	test: 5237.1359804	best: 5237.1359804 (207)	total: 7.44s	remaining: 1.5s
208:	learn: 4691.7091980	test: 5234.9616711	best: 5234.9616711 (208)	total: 7.48s	remaining: 1.47s
209:	learn: 4687.6224928	test: 5231.6948019	best: 5231.6948019 (209)	total: 7.51s	remaining: 1.43s
210:	learn: 4683.6569826	test: 5230.3402453	best: 5230.3402453 (210)	total: 7.55s	remaining: 1.4s
211:	learn: 4680.1689411	test: 5228.8168167	best: 5228.8168167 (211)	total: 7.6s	remaining: 1.36s
212:	learn: 4673.7535400	test: 5227.1309643	best: 5227.1309643 (212)	total: 7.64s	remaining: 1.33s
213:	learn: 4670.0753074	test: 5225.9025252	best: 5225.9025252 (213)	total: 7.68s	remaining: 1.29s
214:	learn: 4665.9597410	test: 5224.2280116	best: 5224.2280116 (214)	total: 7.71s	remaining: 1.25s
215:	learn: 46

41:	learn: 5986.1276260	test: 6150.6444401	best: 6150.6444401 (41)	total: 1.78s	remaining: 8.84s
42:	learn: 5966.8726740	test: 6137.3371626	best: 6137.3371626 (42)	total: 1.82s	remaining: 8.79s
43:	learn: 5948.0479559	test: 6124.0935881	best: 6124.0935881 (43)	total: 1.87s	remaining: 8.74s
44:	learn: 5936.5263411	test: 6111.5516597	best: 6111.5516597 (44)	total: 1.9s	remaining: 8.68s
45:	learn: 5924.9952941	test: 6099.4824357	best: 6099.4824357 (45)	total: 1.94s	remaining: 8.6s
46:	learn: 5913.4113015	test: 6089.6168452	best: 6089.6168452 (46)	total: 1.98s	remaining: 8.54s
47:	learn: 5898.0355948	test: 6079.3188316	best: 6079.3188316 (47)	total: 2.02s	remaining: 8.49s
48:	learn: 5884.1467608	test: 6069.5859854	best: 6069.5859854 (48)	total: 2.06s	remaining: 8.43s
49:	learn: 5871.2581818	test: 6056.2285075	best: 6056.2285075 (49)	total: 2.09s	remaining: 8.37s
50:	learn: 5858.3817747	test: 6046.4145903	best: 6046.4145903 (50)	total: 2.13s	remaining: 8.31s
51:	learn: 5834.5533035	test: 60

127:	learn: 5138.6631716	test: 5569.2439687	best: 5569.2439687 (127)	total: 4.16s	remaining: 3.96s
128:	learn: 5129.7676117	test: 5564.3348918	best: 5564.3348918 (128)	total: 4.18s	remaining: 3.92s
129:	learn: 5122.3240296	test: 5560.6603647	best: 5560.6603647 (129)	total: 4.2s	remaining: 3.88s
130:	learn: 5112.0154732	test: 5556.6232567	best: 5556.6232567 (130)	total: 4.22s	remaining: 3.84s
131:	learn: 5103.8365465	test: 5554.1084082	best: 5554.1084082 (131)	total: 4.26s	remaining: 3.81s
132:	learn: 5098.0803025	test: 5548.8302611	best: 5548.8302611 (132)	total: 4.3s	remaining: 3.78s
133:	learn: 5090.3465830	test: 5545.5614532	best: 5545.5614532 (133)	total: 4.33s	remaining: 3.75s
134:	learn: 5081.4048063	test: 5538.5602669	best: 5538.5602669 (134)	total: 4.37s	remaining: 3.73s
135:	learn: 5074.1017055	test: 5535.6280398	best: 5535.6280398 (135)	total: 4.42s	remaining: 3.7s
136:	learn: 5068.1321430	test: 5532.1213103	best: 5532.1213103 (136)	total: 4.45s	remaining: 3.67s
137:	learn: 5

210:	learn: 4651.4044725	test: 5339.2899243	best: 5339.2899243 (210)	total: 7.39s	remaining: 1.37s
211:	learn: 4643.8396418	test: 5337.2378313	best: 5337.2378313 (211)	total: 7.44s	remaining: 1.33s
212:	learn: 4638.9131002	test: 5335.6908232	best: 5335.6908232 (212)	total: 7.47s	remaining: 1.3s
213:	learn: 4635.1228792	test: 5334.4233905	best: 5334.4233905 (213)	total: 7.51s	remaining: 1.26s
214:	learn: 4631.4892604	test: 5333.6493321	best: 5333.6493321 (214)	total: 7.55s	remaining: 1.23s
215:	learn: 4627.9156862	test: 5331.5864396	best: 5331.5864396 (215)	total: 7.59s	remaining: 1.19s
216:	learn: 4625.2573167	test: 5329.6214298	best: 5329.6214298 (216)	total: 7.63s	remaining: 1.16s
217:	learn: 4622.4732304	test: 5327.4265044	best: 5327.4265044 (217)	total: 7.67s	remaining: 1.13s
218:	learn: 4617.4832918	test: 5325.4958808	best: 5325.4958808 (218)	total: 7.71s	remaining: 1.09s
219:	learn: 4612.7466284	test: 5323.7244157	best: 5323.7244157 (219)	total: 7.75s	remaining: 1.06s
220:	learn:

`Hyper-parameter Tuning`

* r2_score =  0.785446636046637

![R2_score](img.png)

![Overall Model Performance](op2.png)

### Overall performance by the different models,
> **Random Forest Regression** gives the **highest R2_score = 0.818496003833452**

df_check = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_check = df_check.head(25)
#round(df_check,2)
df_check.plot(kind='bar',figsize=(10,5))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.title('Performance of XGBRegressor')
plt.ylabel('Mean Squared Log Error')
plt.savefig('XGBRegressor-Performance.jpg')
plt.show()

`
#Ploting feature importance graph
importances=model.feature_importances_
features=X_train.columns

# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.figure(figsize=(6,6))
plt.bar(x_values, importances, orientation = ('vertical'))
# Tick labels for x axis
plt.xticks(x_values, features, rotation=(90))
# Axis labels and title
plt.ylabel('Importance'); 
plt.xlabel('Variable/Features'); 
plt.title('XGBRegressor Variables Importance')
plt.tight_layout()
plt.savefig('XGBRegressor-Variables-Importances.jpg',dpi=600)
plt.show();`